In [1]:
import numpy as np
import scipy.stats as scps
from scipy.special.orthogonal import ps_roots
import scipy.interpolate as scin
from copy import *
%load_ext autoreload

%autoreload 2

In [2]:
from ret import util, mutil, imutil, income, budget, mbudget, value_function, budget_old, chpr, logsum, egm_step,chop, polyline, diff, aux_function, interpolate, upper_envelope

# Model parameters (default)

In [3]:
# Number of periods (fist period is t=1) 
Tbar = 25

# Number of grid points over assets
ngridm = 500

# Maximum level of assets
mmax = 50

# Number of quadrature points used in calculation of expectations
n_quad_points = 5

# Number of simulations
nsims = 50

# Interval of the initial wealth
init = [10, 30]

# Interest rate
r = 0.05

# Discount factor
beta = 0.95

# Standard deviation of log-normally distributed income shocks
sigma = 0.00

# Disutility of work
duw = 0.35

# CRRA coefficient (log utility if ==1)
theta = 1.95

# Careful with the coefficients here -- original code had the polynomial
# Coded as a + b * x - c * x**2 ... (note the crazy minus)
coeffs_age_poly = np.array([0.75, 0.04, -0.0002])

# Consumption floor (safety net in retirement)
cfloor = 0.001

# Scale of the EV taste shocks 
lambda_ = 2.2204e-16

In [4]:
# Initialize grids
quadstnorm = scps.norm.ppf(ps_roots(n_quad_points)[0])
quadw = ps_roots(n_quad_points)[1]


In [5]:
savingsgrid = np.linspace(0, mmax, ngridm)

In [6]:
# We are currently facing a problem since our result arrays do not have the same length. 
# Instead of a multidimensional array we should use a list in which we store the different arrays.
# What would this mean for our functions? 

In [7]:
# Set up list containers
policy = [[np.full((ngridm + 1, 2), np.nan) for k in range(2)] for i in range(Tbar)]
value = [[np.full((ngridm + 1, 2), np.nan) for k in range(2)] for i in range(Tbar)]
# Handling of last period and first elements
# policy
for k in range(Tbar):
    for choice in range(2):
        value[k][choice][0, 0] = 0.00
        policy[k][choice][0, :] = 0.00

policy[Tbar - 1][0][1:, 0] = deepcopy(savingsgrid)
policy[Tbar-1][1][1:, 0] = deepcopy(savingsgrid)
policy[Tbar-1][0][1:, 1] = deepcopy(policy[Tbar-1][0][1:, 0])
policy[Tbar-1][1][1:, 1] = deepcopy(policy[Tbar-1][1][1:, 0])
# value
value[Tbar-1][0][2:, 0] = util(policy[Tbar-1][0][2:, 0], 0, theta, duw)
value[Tbar-1][0][2:, 1] = util(policy[Tbar-1][1][2:, 0], 1, theta, duw)
value[Tbar-1][1][2:, 0] = util(policy[Tbar-1][0][2:, 0], 0, theta, duw)
value[Tbar-1][1][2:, 1] = util(policy[Tbar-1][1][2:, 0], 1, theta, duw)

value[Tbar -1][0][0:2] = 0.00
value[Tbar -1][1][0:2] = 0.00

# The time and the choice dimension are now extracted from our array format. 
# Instead we are using a list for saving the results which allows us to alter the legth of the arrays


In [10]:
for period in [23, 22, 21, 20, 19]:
    value, policy = egm_step(value, policy, savingsgrid, quadstnorm, period, Tbar, ngridm, cfloor, n_quad_points, r, coeffs_age_poly, theta, duw, beta, lambda_, sigma, quadw)

In [11]:
import pickle
with open('m0_value.pkl', 'rb') as file : 
    m0_value = pickle.load(file)

with open('m0_policy.pkl', 'rb') as file : 
    m0_policy = pickle.load(file)


In [12]:
    

for point in [23, 22, 21, 20, 19]:
    np.testing.assert_almost_equal(m0_value[:, 0, 1, point].T[~np.isnan(m0_value[:, 0, 1, point])], value[point][1].T[0], decimal=7)
    np.testing.assert_almost_equal(m0_value[:, 1, 1, point].T[~np.isnan(m0_value[:, 1, 1, point])], value[point][1].T[1], decimal=7)
    np.testing.assert_almost_equal(m0_policy[:, 0, 1, point].T[~np.isnan(m0_policy[:, 0, 1, point])], policy[point][1].T[0], decimal=7)
    np.testing.assert_almost_equal(m0_policy[:, 1, 1, point].T[~np.isnan(m0_policy[:, 1, 1, point])], policy[point][1].T[1], decimal=7)
    np.testing.assert_almost_equal(m0_value[:, 0, 0, point].T[~np.isnan(m0_value[:, 0, 0, point])], value[point][0].T[0], decimal=7)
    np.testing.assert_almost_equal(m0_value[:, 1, 0, point].T[~np.isnan(m0_value[:, 1, 0, point])], value[point][0].T[1], decimal=7)
    np.testing.assert_almost_equal(m0_policy[:, 0, 0, point].T[~np.isnan(m0_policy[:, 0, 0, point])], policy[point][0].T[0], decimal=7)
    np.testing.assert_almost_equal(m0_policy[:, 1, 0, point].T[~np.isnan(m0_policy[:, 1, 0, point])], policy[point][0].T[1], decimal=7)


23
22
21
20
19


In [ ]:
value, policy = egm_step(value, policy, savingsgrid, quadstnorm, 18, Tbar, ngridm, cfloor, n_quad_points, r, coeffs_age_poly, theta, duw, beta, lambda_, sigma, quadw)

## Differences between policy and the m0policy values for working == 1 

In [15]:
policy[19][1].T[0] - m0_policy[:, 0, 1, 19].T[~np.isnan(m0_policy[:, 0, 1, 19])]

array([ 0.00000000e+00, -4.79616347e-14, -5.15143483e-14, -5.41788836e-14,
       -5.77315973e-14, -4.88498131e-14, -5.41788836e-14, -5.59552404e-14,
       -5.95079541e-14, -5.24025268e-14, -5.41788836e-14, -5.68434189e-14,
       -6.03961325e-14, -5.41788836e-14, -5.77315973e-14, -5.95079541e-14,
       -5.24025268e-14, -5.50670620e-14, -5.77315973e-14, -6.12843110e-14,
       -5.32907052e-14, -5.50670620e-14, -5.86197757e-14, -6.21724894e-14,
       -5.41788836e-14, -5.77315973e-14, -6.03961325e-14, -6.39488462e-14,
       -6.75015599e-14,  2.82440737e-13, -1.59872116e-13, -4.26325641e-14,
       -4.61852778e-14, -4.79616347e-14, -4.08562073e-14, -4.44089210e-14,
       -4.97379915e-14, -4.97379915e-14, -4.08562073e-14, -4.61852778e-14,
       -4.61852778e-14, -4.97379915e-14, -5.50670620e-14, -4.79616347e-14,
       -4.97379915e-14, -3.37507799e-14, -5.50670620e-14, -7.63833441e-14,
       -9.94759830e-14, -2.30926389e-14, -4.79616347e-14, -7.10542736e-14,
       -9.23705556e-14, -

In [16]:
policy[19][1].T[1] - m0_policy[:, 1, 1, 19].T[~np.isnan(m0_policy[:, 0, 1, 19])]

array([ 0.00000000e+00, -4.79616347e-14, -5.50670620e-14, -5.06261699e-14,
       -5.68434189e-14, -5.24025268e-14, -6.03961325e-14, -5.50670620e-14,
       -5.24025268e-14, -5.77315973e-14, -5.32907052e-14, -5.95079541e-14,
       -5.59552404e-14, -5.24025268e-14, -5.86197757e-14, -5.50670620e-14,
       -6.03961325e-14, -5.59552404e-14, -6.30606678e-14, -5.86197757e-14,
       -5.41788836e-14, -5.95079541e-14, -5.59552404e-14, -6.30606678e-14,
       -5.77315973e-14, -5.50670620e-14, -6.03961325e-14, -5.77315973e-14,
       -6.39488462e-14, -3.46389584e-14, -3.10862447e-14, -4.44089210e-14,
       -5.06261699e-14, -4.61852778e-14, -4.26325641e-14, -4.88498131e-14,
       -4.61852778e-14, -5.24025268e-14, -4.61852778e-14, -4.17443857e-14,
       -4.70734562e-14, -4.44089210e-14, -5.15143483e-14, -4.70734562e-14,
       -5.32907052e-14, -4.88498131e-14, -5.50670620e-14, -5.06261699e-14,
       -5.59552404e-14, -5.32907052e-14, -4.97379915e-14, -5.59552404e-14,
       -5.24025268e-14, -

In [29]:
wk1 = budget(
    18,
    savingsgrid,
    quadstnorm * sigma,
    1.,
    ngridm,
    n_quad_points,
    r,
    coeffs_age_poly,
)
cons11_interpolate = scin.interp1d(
    policy[19][1].T[0],
    policy[19][1].T[1],
    bounds_error=False,
    fill_value="extrapolate",
)
cons11_flat = cons11_interpolate(wk1).flatten("F")

cons11_interpolate = scin.interp1d(
    m0_policy[:, 0, 1, 19].T[~np.isnan(m0_policy[:, 0, 1, 19])],
    m0_policy[:, 1, 1, 19].T[~np.isnan(m0_policy[:, 1, 1, 19])],
    bounds_error=False,
    fill_value="extrapolate",
)
cons11_flat_org = cons11_interpolate(wk1).flatten("F")


In [39]:
np.where(cons11_flat - cons11_flat_org > 0.01)

(array([ 45,  46,  47,  48,  49, 360, 361, 362, 363, 364]),)